In [1]:
import openai

In [3]:
import praw

In [4]:
reddit = praw.Reddit(client_id = 'qWxOfO8sO1E08gYjMd_c1g',
                    client_secret = 'I3j54HiXEzQTJ262giFgaH3V1jI5pg',
                    user_agent = "sentiment_analysis_test")

In [9]:
import os
os.environ['REDDIT_CLIENT_ID'] = 'qWxOfO8sO1E08gYjMd_c1g'
os.environ['REDDIT_SECRET'] = 'I3j54HiXEzQTJ262giFgaH3V1jI5pg'

In [7]:
for submission in reddit.subreddit("openai").hot(limit=5):
    print(submission.title)

We just hit 100k! Who wants to run r/OpenAI?
Revenge.
Created an application that generate memes with a single click! Using GPT-4 and BLIP-2
"Verify you are human" infinite loop
Yo, Openai just gave me a 2nd Gpt-4 model. Anyone else? I'm guessing this could be the longer context model, the vision model or just a bug.


In [29]:
subreddit_topic = reddit.subreddit("stocks")

In [30]:
subreddit_topic.accounts_active

5958

In [31]:
for post in subreddit_topic.hot(limit=5):
    print(post.title)
    submission = reddit.submission(post.id)
    # print top 2 comments per title submission
    counter = 0
    for comment in submission.comments:
        print(comment.body)
        counter += 1
        if counter == 2:
            break

Rate My Portfolio - r/Stocks Quarterly Thread March 2023
I am 26, so I have several decades until retirement. My goal is to hold the stocks in my portfolio perpetually, though I'm not reserved to make adjustments and sell when a) I see an opportunity to take profits b) my thesis changes. My long term goal is passive income generation, though I don't chase yields. I don't pay much attention to valuation or macro, as I think that since I plan on holding these stocks for decades while buying every month, those don't matter to me much. I invest regularly every month, so I'm okay with buying at a higher valuation at times and lower valuation at other times. In my opinion, buying at the best possible time is more difficult than buying a good company. That said, if I see an undervalued stock I like, I don't hesitate to buy more and when I make my regular contributions, I try weigh towards stocks that might have been beaten down or seem undervalued. My last review in one of these was [Dec 2021

In [47]:
def get_titles_and_comments(subreddit="stocks", limit=6, num_comments=3, skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    title_and_comments = {}
    
    for counter, post in enumerate(subreddit.hot(limit=limit)):
        if counter < skip_first:
            continue
            
        counter += (1-skip_first)
        
        title_and_comments[counter] = ""
        # PRAW
        submission = reddit.submission(post.id)
        title = post.title
        
        title_and_comments[counter] += "Title: "+title+"\n\n"
        title_and_comments[counter] += "Comments: \n\n"
        # {0: "Title: Post Title \n\nComments: \n\n"}
        
        comment_counter = 0
        for comment in submission.comments:
            if not comment.body == "[deleted]":
                title_and_comments[counter] += comment.body+"\n"
                comment_counter += 1
            if comment_counter == num_comments:
                break
    
    return title_and_comments

In [48]:
title_and_comments = get_titles_and_comments()

In [49]:
title_and_comments

{1: 'Title: Apple’s “Wearables & Accessories” revenue ($41.24B) is significantly greater than Nvidia’s annual revenue (26.91B).\n\nComments: \n\nComparing revenue is pointless.\n\nWalmart has 611bn revenue vs apple’s 394bn.\n\nWalmart is 396bn mcap vs apples 2620bn.\n\nEveryone will now jump in about margins and profit. Which is the point im trying to make.\n\nPeople might also talk about profits being in apple’s favor \n\nThen i should bring up exxon mobil that had profits of 77bn off 398bn revenue.\n\nTrading at only 468bn market cap\n\nPeople might now talk about different industry, growth potential, cyclical businesses etc.\n\nNow people should understand comparing nvda to apple wearables by looking at revenue is completely useless\nSo buy NVDA and AAPL, got it.\nBut apple has no AI hype\n',
 2: 'Title: ‘I am truly sorry’: Credit Suisse chair pleads with angry shareholders at annual meeting\n\nComments: \n\nThe fact his last name is “Lehmann” is just…fitting.\nWe’re *sorry*.  We re

In [50]:
def create_prompt(title_and_comments):
    task = "Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive, negative, or neutral. If no ticker or company is mentioned write 'No company mentioned'\n\n"
    return task+title_and_comments

In [51]:
print(create_prompt(title_and_comments[1]))

Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive, negative, or neutral. If no ticker or company is mentioned write 'No company mentioned'

Title: Apple’s “Wearables & Accessories” revenue ($41.24B) is significantly greater than Nvidia’s annual revenue (26.91B).

Comments: 

Comparing revenue is pointless.

Walmart has 611bn revenue vs apple’s 394bn.

Walmart is 396bn mcap vs apples 2620bn.

Everyone will now jump in about margins and profit. Which is the point im trying to make.

People might also talk about profits being in apple’s favor 

Then i should bring up exxon mobil that had profits of 77bn off 398bn revenue.

Trading at only 468bn market cap

People might now talk about different industry, growth potential, cyclical businesses etc.

Now people should understand comparing nvda to apple wearables by looking at revenue is completely useless
So buy NVDA and AAPL, got it.
But apple has n

In [52]:
for key, title_with_comments in title_and_comments.items():
    prompt = create_prompt(title_with_comments)
    
    response = openai.Completion.create(
        engine = 'text-davinci-003',
        prompt = prompt,
        max_tokens = 256,
        temperature = 0,
        top_p = 1.0
    )
    
    print(title_with_comments)
    print(f"Sentiment Report from OpenAI: {response['choices'][0]['text']}")
    print('-----------------------------')

Title: Apple’s “Wearables & Accessories” revenue ($41.24B) is significantly greater than Nvidia’s annual revenue (26.91B).

Comments: 

Comparing revenue is pointless.

Walmart has 611bn revenue vs apple’s 394bn.

Walmart is 396bn mcap vs apples 2620bn.

Everyone will now jump in about margins and profit. Which is the point im trying to make.

People might also talk about profits being in apple’s favor 

Then i should bring up exxon mobil that had profits of 77bn off 398bn revenue.

Trading at only 468bn market cap

People might now talk about different industry, growth potential, cyclical businesses etc.

Now people should understand comparing nvda to apple wearables by looking at revenue is completely useless
So buy NVDA and AAPL, got it.
But apple has no AI hype

Sentiment Report from OpenAI: 
Apple: Neutral
-----------------------------
Title: ‘I am truly sorry’: Credit Suisse chair pleads with angry shareholders at annual meeting

Comments: 

The fact his last name is “Lehmann” is